In [26]:
pip install imagecodecs

  Using cached imagecodecs-2025.3.30-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (45.6 MB)

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [32]:
pip install rasterio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.2/22.2 MB 11.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 13.6 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-model-optimization 0.7.4 requires numpy~=1.23, but you have numpy 2.2.6 which is incompatible.
scipy 1.10.1 requires numpy<1.27.0,>=1.19.5, but you have numpy 2.2.6 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 2.2.6 which is incompatible.

[notice] A new release of pip is available: 23.1.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install --upgrade --force-reinstall numpy matplotlib


  Using cached numpy-2.2.6-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (16.8 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 12.2 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 326.2/326.2 kB 1.3 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 8.4 MB/s eta 0:00:00:00:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.4 MB/s eta 0:00:00:00:01
  Using cached packaging-25.0-py3-none-any.whl (66 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.6/4.6 MB 12.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.1/111.1 kB 687.8 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 599.6 kB/s eta 0:00:0000:01
  Attempting uninstall: six
    Found existing installation: six 1.16.0
    Uninstalling six-1.16.0:
      Successfully uninstalled six-1.16.0
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.0.9
    Un

In [ ]:
!pip install "numpy<2"

In [ ]:
!pip install scikit-learn

In [ ]:
import os
import re
import json
import random
from pathlib import Path
import xml.etree.ElementTree as ET
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import tifffile as tiff
import openslide
import rasterio
from rasterio.windows import Window
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, ConcatDataset
from torchvision import models, transforms, datasets
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix


In [2]:
happiny_folder = Path("/data/temporary/archives/lung/generator/MONKEY_challenge")
orig_dir = happiny_folder / "images/pas-original"
cpg_dir = happiny_folder / "images/pas-cpg"
diagnostic_dir = happiny_folder / "images/pas-diagnostic"
xml_dir = happiny_folder / "annotations/xml"

# Extracting patches from ROIs from the train dirs

In [ ]:
PATCH_SIZE = 256
NUM_JSONS = 10
NUM_PATCHES_PER_FILE = 10

happiny_folder = Path("/data/temporary/archives/lung/generator/MONKEY_challenge")
json_pixel_dir = happiny_folder / "annotations/json_pix"
PATCHES_DIR = happiny_folder / "patches"  
modalities = {
    "pas-original": happiny_folder / "images/pas-original",
    "pas-cpg": happiny_folder / "images/pas-cpg",
    "pas-diagnostic": happiny_folder / "images/pas-diagnostic",
}

PATCHES_DIR.mkdir(parents=True, exist_ok=True)

def get_sample_id(json_path):
    match = re.search(r'P\d{6}', json_path.name)
    return match.group(0) if match else None

def load_json_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return [(int(p["point"][0]), int(p["point"][1])) for p in data.get("points", [])]

def extract_patch(tif_path, center_x, center_y, patch_size=PATCH_SIZE):
    half = patch_size // 2
    xmin = max(center_x - half, 0)
    ymin = max(center_y - half, 0)
    with rasterio.open(tif_path) as src:
        max_width, max_height = src.width, src.height
        xmin = min(xmin, max_width - patch_size)
        ymin = min(ymin, max_height - patch_size)
        window = Window(xmin, ymin, patch_size, patch_size)
        patch = src.read(window=window)
        patch = np.transpose(patch, (1, 2, 0))  # (H, W, C)
    return patch

def save_selected_patches():
    json_files = list(json_pixel_dir.glob("*inflammatory-cells.json"))
    print(f"🔍 Found {len(json_files)} inflammatory cell JSON files")

    # Randomly sample JSON files without replacement
    json_files_sampled = random.sample(json_files, min(NUM_JSONS, len(json_files)))

    for modality, tif_dir in modalities.items():
        output_dir = PATCHES_DIR / modality
        output_dir.mkdir(parents=True, exist_ok=True)
        print(f"\nProcessing modality: {modality}")

        for json_file in json_files_sampled:
            coords = load_json_annotations(json_file)[:NUM_PATCHES_PER_FILE]
            sample_id = get_sample_id(json_file)
            if not sample_id:
                continue

            tif_matches = list(tif_dir.glob(f"*{sample_id}*.tif"))
            if not tif_matches:
                print(f"  ⚠️ No TIFF file found for sample ID {sample_id} in {modality}")
                continue

            tif_path = tif_matches[0]  # Take the first match
            print(f"  Using {tif_path.name} with {len(coords)} ROIs")

            for i, (x, y) in enumerate(coords):
                try:
                    patch = extract_patch(tif_path, x, y, PATCH_SIZE)
                    patch_img = Image.fromarray(patch.astype(np.uint8))
                    patch_name = f"{sample_id}_patch{i}.png"
                    patch_img.save(output_dir / patch_name)
                except Exception as e:
                    print(f"    ⚠️ Failed to extract patch {i} at ({x},{y}): {e}")

if __name__ == "__main__":
    save_selected_patches()
    print("\n✅ Patch extraction complete.")


🔍 Found 81 inflammatory cell JSON files

Processing modality: pas-original
  ⚠️ No TIFF file found for sample ID P000021 in pas-original
  ⚠️ No TIFF file found for sample ID P000021 in pas-original
  Using D_P000019_PAS_Original.tif with 10 ROIs
  Using D_P000016_PAS_Original.tif with 10 ROIs
  Using D_P000003_PAS_Original.tif with 10 ROIs
  Using B_P000020_PAS_Original.tif with 10 ROIs
  Using D_P000002_PAS_Original.tif with 10 ROIs
  ⚠️ No TIFF file found for sample ID P000033 in pas-original
  Using D_P000006_PAS_Original.tif with 10 ROIs
  Using D_P000002_PAS_Original.tif with 10 ROIs

Processing modality: pas-cpg
  Using A_P000021_PAS_CPG.tif with 10 ROIs
  Using A_P000021_PAS_CPG.tif with 10 ROIs
  Using D_P000019_PAS_CPG.tif with 10 ROIs
  Using D_P000016_PAS_CPG.tif with 10 ROIs
  Using B_P000003_PAS_CPG.tif with 10 ROIs
  Using A_P000020_PAS_CPG.tif with 10 ROIs
  Using D_P000002_PAS_CPG.tif with 10 ROIs
  Using A_P000033_PAS_CPG.tif with 10 ROIs
  Using D_P000006_PAS_CPG.tif

# Extracting patches from ROIs from the test dirs

In [ ]:
# Configuration
PATCH_SIZE = 256
NUM_PATCHES_PER_FILE = 5

# Paths
happiny_folder = Path("/data/temporary/archives/lung/generator/MONKEY_challenge")
json_pixel_dir = happiny_folder / "annotations/json_pix"
PATCHES_DIR = happiny_folder / "patches"

cpg_dir = happiny_folder / "images/pas-cpg"

# Create output directories for cpg modalities + cell types
for cell_type in ["inflammatory-cells", "lymphocytes", "monocytes"]:
    (PATCHES_DIR / "cpg" / cell_type).mkdir(parents=True, exist_ok=True)

# Helpers
def get_sample_id(json_path):
    match = re.search(r'P\d{6}', json_path.name)
    return match.group(0) if match else None

def load_json_annotations(json_path):
    with open(json_path, 'r') as f:
        data = json.load(f)
    return [(int(p["point"][0]), int(p["point"][1])) for p in data.get("points", [])]

def extract_patch(tif_path, center_x, center_y, patch_size=PATCH_SIZE):
    half = patch_size // 2
    xmin = max(center_x - half, 0)
    ymin = max(center_y - half, 0)
    with rasterio.open(tif_path) as src:
        max_width, max_height = src.width, src.height
        xmin = min(xmin, max_width - patch_size)
        ymin = min(ymin, max_height - patch_size)
        window = Window(xmin, ymin, patch_size, patch_size)
        patch = src.read(window=window)
        patch = np.transpose(patch, (1, 2, 0))  # (H, W, C)
    return patch

def save_cpg_patches_for_cell_types():
    # Cell types to process
    cell_types = ["inflammatory-cells", "lymphocytes", "monocytes"]
    
    # For each cell type, find relevant json files
    for cell_type in cell_types:
        json_files = sorted(json_pixel_dir.glob(f"*{cell_type}.json"))
        print(f"Found {len(json_files)} {cell_type} JSON files")

        for json_file in json_files:
            coords = load_json_annotations(json_file)[:NUM_PATCHES_PER_FILE]
            sample_id = get_sample_id(json_file)
            if not sample_id:
                continue

            tif_matches = list(cpg_dir.glob(f"*{sample_id}*.tif"))
            if not tif_matches:
                continue

            tif_path = tif_matches[0]  # Use the first match
            print(f"Processing {tif_path.name} with {len(coords)} {cell_type} ROIs")

            output_dir = PATCHES_DIR / "cpg" / cell_type

            for i, (x, y) in enumerate(coords):
                patch = extract_patch(tif_path, x, y, PATCH_SIZE)
                patch_img = Image.fromarray(patch.astype(np.uint8))
                patch_name = f"{sample_id}_{cell_type}_patch{i}.png"
                patch_img.save(output_dir / patch_name)

if __name__ == "__main__":
    save_cpg_patches_for_cell_types()
    print("✅ Finished extracting cpg patches for inflammatory, lymphocytes, and monocytes.")


Found 81 inflammatory-cells JSON files
Processing B_P000001_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing D_P000002_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing B_P000003_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing D_P000004_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing A_P000005_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing D_P000006_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing A_P000007_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing B_P000011_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing D_P000014_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing D_P000016_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing A_P000017_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing A_P000018_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing A_P000020_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing A_P000021_PAS_CPG.tif with 5 inflammatory-cells ROIs
Processing C_P000022_PAS_CPG.tif with 5 inflammatory-cells ROIs
P

# Model building

In [ ]:
train_dirs = [
    PATCHES_DIR / "pas-original",
    PATCHES_DIR / "pas-diagnostic"
]
test_dirs = [
    PATCHES_DIR / "cpg" / "inflammatory-cells",
    PATCHES_DIR / "cpg" / "monocytes",
    PATCHES_DIR / "cpg" / "lymphocytes"
]
cpg_inflammatory_dir = PATCHES_DIR / "cpg" / "inflammatory-cells"

class_names_all = ["inflammatory-cells", "monocytes", "lymphocytes"]

transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

def wrap_with_label_folder(path, label="dummy"):
    temp = path.parent / f"{path.name}_wrapped"
    temp.mkdir(exist_ok=True)
    label_path = temp / label
    label_path.mkdir(exist_ok=True)
    for img in path.glob("*.png"):
        symlink = label_path / img.name
        if not symlink.exists():
            symlink.symlink_to(img)
    return temp

# Create dataloaders from directories
def create_dataloader(dirs, transform, batch_size=32, shuffle=True):
    wrapped_dirs = [wrap_with_label_folder(d) for d in dirs]
    datasets_list = [datasets.ImageFolder(d, transform=transform) for d in wrapped_dirs]
    combined_dataset = ConcatDataset(datasets_list)
    loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=shuffle)
    return loader

# Extract features for a given dataset loader and encoder model
def extract_features(encoder, loader, device):
    encoder.eval()
    all_feats = []
    all_labels = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            feats = encoder(imgs).cpu()
            all_feats.append(feats)
            all_labels.append(labels)
    X = torch.cat(all_feats).numpy()
    y = torch.cat(all_labels).numpy()
    return X, y

# Pretrain encoder on train_dirs (pas-original + pas-diagnostic)
def pretrain_encoder(train_dirs, transform, device, epochs=5):
    train_loader = create_dataloader(train_dirs, transform, batch_size=32, shuffle=True)
    encoder = models.resnet18(pretrained=True)
    encoder.fc = nn.Identity()  
    encoder = encoder.to(device)
    optimizer = optim.Adam(encoder.parameters(), lr=1e-4)
    encoder.train()
    for epoch in range(epochs):
        for imgs, _ in tqdm(train_loader, desc=f"Pretraining Epoch {epoch+1}"):
            imgs = imgs.to(device)
            features = encoder(imgs)
            loss = features.norm(dim=1).mean() 
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return encoder

# Fine-tune encoder on only CPG inflammatory-cells (binary classification)
def finetune_on_cpg_inflammatory(encoder, cpg_dir, transform, device, epochs=3):
    # Prepare dataset with label 0 for inflammatory cells (binary classification)
    wrapped_dir = wrap_with_label_folder(cpg_dir, label="inflammatory-cells")
    dataset = datasets.ImageFolder(wrapped_dir, transform=transform)
    loader = DataLoader(dataset, batch_size=32, shuffle=True)
    
    # Add classification head for binary classification
    encoder.fc = nn.Linear(512, 2)
    encoder = encoder.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(encoder.parameters(), lr=1e-4)
    
    encoder.train()
    for epoch in range(epochs):
        for imgs, labels in tqdm(loader, desc=f"Fine-tuning CPG inflammatory Epoch {epoch+1}"):
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = encoder(imgs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
    return encoder

# Prepare test dataloader for all CPG classes with correct labels
def create_cpg_test_loader(test_dirs, transform, batch_size=32):
    wrapped_dirs = []
    for i, d in enumerate(test_dirs):
        wrapped_dirs.append(wrap_with_label_folder(d, label=str(i)))  # label folder name = class index
    datasets_list = [datasets.ImageFolder(d, transform=transform) for d in wrapped_dirs]
    combined_dataset = ConcatDataset(datasets_list)
    loader = DataLoader(combined_dataset, batch_size=batch_size, shuffle=False)
    return loader

# Extract features with fine-tuned encoder for multi-class classification
def extract_features_multiclass(encoder, loader, device):
    encoder.eval()
    all_feats = []
    all_labels = []
    with torch.no_grad():
        for imgs, labels in loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            feats = encoder(imgs)
            all_feats.append(feats.cpu())
            all_labels.append(labels.cpu())
    X = torch.cat(all_feats).numpy()
    y = torch.cat(all_labels).numpy()
    return X, y

# --------- MAIN PIPELINE ---------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Model A: Pretrain encoder on pas-original + pas-diagnostic only
print("Pretraining encoder (Model A)...")
encoder_A = pretrain_encoder(train_dirs, transform, device)

# Test Model A on all CPG classes with RF classifier on frozen encoder features
print("\nEvaluating Model A (no fine-tuning on CPG)...")
cpg_test_loader = create_cpg_test_loader(test_dirs, transform, batch_size=32)
X_A, y_A = extract_features(encoder_A, cpg_test_loader, device)

clf_A = RandomForestClassifier(n_estimators=100, random_state=42)
clf_A.fit(X_A, y_A)
y_pred_A = clf_A.predict(X_A)

print("Model A Confusion Matrix:")
print(confusion_matrix(y_A, y_pred_A))
print("Model A Classification Report:")
print(classification_report(y_A, y_pred_A, target_names=class_names_all))

# Model B: Fine-tune encoder on CPG inflammatory-cells (binary) 
print("\nFine-tuning encoder on CPG inflammatory-cells (Model B)...")
encoder_B = finetune_on_cpg_inflammatory(encoder_A, cpg_inflammatory_dir, transform, device, epochs=3)

# For Model B testing, remove classification head for feature extraction (to get embeddings)
encoder_B.fc = nn.Identity()
encoder_B = encoder_B.to(device)

# Extract features from all CPG classes with fine-tuned encoder
print("\nEvaluating Model B (fine-tuned on CPG inflammatory-cells)...")
X_B, y_B = extract_features(encoder_B, cpg_test_loader, device)

clf_B = RandomForestClassifier(n_estimators=100, random_state=42)
clf_B.fit(X_B, y_B)
y_pred_B = clf_B.predict(X_B)

print("Model B Confusion Matrix:")
print(confusion_matrix(y_B, y_pred_B))
print("Model B Classification Report:")
print(classification_report(y_B, y_pred_B, target_names=class_names_all))
